# Acompanhamento - Atividades Jira

## GRUPO SEI

---

### Informações iniciais

**Edite apenas aqui!**

In [26]:
# --------------------------------------------------
# SPRINT A ANALISAR
# --------------------------------------------------
SPRINT = "04"
DIAS_UTEIS = 13 # Numero de dias uteis, sem contar primeiro e ultimo dia de sprint e feriados
FILTER_ID = 10484



# --------------------------------------------------
# Jira
# Gere seu token pessoal aqui: https://id.atlassian.com/manage-profile/security/api-tokens
# --------------------------------------------------
import json
with open('AtlassianToken.json') as user_file:
  token = json.load(user_file)

user = token["user"]
authToken = token['authToken']
# --------------------------------------------------

### Bibliotecas

In [27]:
from datetime import date, datetime, timedelta
from plotly.subplots import make_subplots
import matplotlib.pyplot as plt
import numpy as np
import requests
import plotly
import plotly.graph_objects as go

### Coleta de dados via Jira API

In [28]:
# DATAS
sprints = {
    "01":{"startday":"10-01-2023", "endday":"27-01-2023"},
    "02":{"startday":"30-01-2023", "endday":"17-02-2023"},
    "03":{"startday":"20-02-2023", "endday":"10-03-2023"},
    "04":{"startday":"13-03-2023", "endday":"31-03-2023"},
    "05":{"startday":"03-04-2023", "endday":"21-04-2023"},
    "06":{"startday":"24-04-2023", "endday":"--2023"},
    "07":{"startday":"--2023", "endday":"--2023"},
    "08":{"startday":"--2023", "endday":"--2023"},
    "09":{"startday":"--2023", "endday":"--2023"},
    "10":{"startday":"--2023", "endday":"--2023"},
    "11":{"startday":"--2023", "endday":"--2023"},
    "12":{"startday":"--2023", "endday":"--2023"},
    "13":{"startday":"--2023", "endday":"--2023"},
    "14":{"startday":"--2023", "endday":"--2023"},
    "15":{"startday":"--2023", "endday":"--2023"},
    "16":{"startday":"--2023", "endday":"--2023"},
    "17":{"startday":"--2023", "endday":"--2023"},
}


HOJE = date.today().strftime("%d-%m-%Y")
DATA_INICIAL = sprints[SPRINT]["startday"]
DATA_FINAL = sprints[SPRINT]["endday"]
startday = datetime.strptime(DATA_INICIAL, "%d-%m-%Y")
endday = datetime.strptime(DATA_FINAL, "%d-%m-%Y")
today = datetime.strptime(HOJE, "%d-%m-%Y")

# --------------------------------------------------
# COLETA DE DADOS VIA API
# --------------------------------------------------
url = f'https://cnpem.atlassian.net/sr/jira.issueviews:searchrequest-csv-all-fields/{FILTER_ID}/SearchRequest-{FILTER_ID}.csv?atl_token=50900f97-f628-45cb-a7f4-b46d9a88f7c9_0830deba9bb0b468e2fa6057a022c670d36e43c7_lin'
download = requests.get(url, auth=(f'{user}', f'{authToken}'))
csv = download.content.decode('utf-8')
# --------------------------------------------------



# --------------------------------------------------
# Salvar em arquivo
# Se ultimo dia do sprint: salva em arquivo final
# Se dia intermediario, salva/sobrescreve arquivo de acompanhamento
# --------------------------------------------------
if (today <= endday):
    f = open("Sprint{}/csv/Sprint_{}_{}.csv".format(SPRINT, SPRINT, HOJE), "w")
    f.write(csv)
    f.close()
# --------------------------------------------------

### Leitura do CSV

In [29]:
# --------------------------------------------------
# DATAS
# --------------------------------------------------
HOJE = date.today().strftime("%d-%m-%Y")   
startday = datetime.strptime(DATA_INICIAL, "%d-%m-%Y")
endday = datetime.strptime(DATA_FINAL, "%d-%m-%Y")
today = datetime.strptime(HOJE, "%d-%m-%Y")


# --------------------------------------------------
# Leitura do CSV
# --------------------------------------------------
if (today >= endday):
    linhas = open("Sprint{}/csv/Sprint_{}_{}.csv".format(SPRINT, SPRINT, DATA_FINAL),'r').read()
else:
    linhas = open("Sprint{}/csv/Sprint_{}_{}.csv".format(SPRINT, SPRINT, HOJE),'r').read()

fields = (linhas.split("Status Category\n")[0]+"Status Category").split(',')

linhas = linhas.split("Status Category\n")[1]
linhas = linhas.replace("Done\n","Done\n\r\r\r").replace("In Progress\n","In Progress\n\r\r\r").replace("To Do\n","To Do\n\r\r\r")
linhas = linhas.replace(", "," ")
linhas = linhas.replace(",height",";")
atividade = linhas.split("\n\r\r\r")[:-1]
# --------------------------------------------------

### Tratamento dos dados e informações a coletar

In [30]:
# --------------------------------------------------
# Variáveis a analisar
# --------------------------------------------------
projetos = {}
totalPontos = 0
membros = {
            "Bruno":{"pontos":0, "natividades":0, "id":"63d01fd6c565900ff40519b9"},
            "Eduarda":{"pontos":0, "natividades":0, "id":"62bb04adde439ce00ee39aa9"},
            "Gabriel":{"pontos":0, "natividades":0, "id":"70121:016a6576-395b-4462-b1e7-fb210bae9d77"},
            "Patricia":{"pontos":0, "natividades":0, "id":"62b0acb8f38b4dcf73d839ec"},
            "Ramon":{"pontos":0, "natividades":0, "id":"62e90951e50f2f2a39548aa1"},
            "Robert":{"pontos":0, "natividades":0, "id":"62bf029efbc1f7c647b9f327"},
            "Rogério":{"pontos":0, "natividades":0, "id":"62bb024e018dea0d4073dbdd"},
            "Samuel":{"pontos":0, "natividades":0, "id":"62b0928ba58208122d9fbcb5"},
          }

status = {
            "Done":{"pontos":0, "natividades":0},
            "In Progress":{"pontos":0, "natividades":0},
            "To Do":{"pontos":0, "natividades":0}
         }

scrum_days = {}
for i in range(15):
    scrum_days[startday + timedelta(days=(7*int(i/5)+(i%5)))] = {"nday":i+1,
                                                                  "points":0,
                                                                  "integrated points":0}

# --------------------------------------------------
# TRATAMENTO DOS DADOS - POR ATIVIDADE (Cada atividade é um item da lista)
# --------------------------------------------------
for i in range(len(atividade)):
    proj = atividade[i].split(",")[fields.index("Project key")]
    if proj not in projetos:
        projetos[proj] = {"name":atividade[i].split(",")[fields.index("Project name")],
                         "totalPoints":0}
    #print(len(atividade[i].split(",")))
    totalPontos += float(atividade[i].split(",")[fields.index("Custom field (Story Points)")])
    projetos[proj]["totalPoints"] += float(atividade[i].split(",")[fields.index("Custom field (Story Points)")])
    
    for pessoa in membros.keys():
        if (pessoa in atividade[i].split(",")[fields.index("Assignee")]) or (membros[pessoa]["id"] in atividade[i].split(",")[fields.index("Description")]):
            membros[pessoa]['pontos'] += float(atividade[i].split(",")[fields.index("Custom field (Story Points)")])
            membros[pessoa]['natividades'] += 1

    for stat in status.keys():
        if (stat in atividade[i].split(",")[fields.index("Status Category")]):
            status[stat]['pontos'] += float(atividade[i].split(",")[fields.index("Custom field (Story Points)")])
            status[stat]['natividades'] += 1
     
    if atividade[i].split(",")[fields.index("Resolved")] != "":
        resolved = datetime.strptime(atividade[i].split(",")[fields.index("Resolved")].split(" ")[0], "%d/%b/%y")
        try:
            scrum_days[resolved]["points"] += float(atividade[i].split(",")[fields.index("Custom field (Story Points)")])
        except:
            continue
# --------------------------------------------------
    
    
# --------------------------------------------------
# PONTOS INTEGRADOS - BURNUP
# --------------------------------------------------
for day in scrum_days.keys():
    if (day == startday):
        scrum_days[day]["integrated points"] = scrum_days[day]["points"]
    else:
        previous_day = list(scrum_days.keys())[scrum_days[day]["nday"]-2]
        scrum_days[day]["integrated points"] = scrum_days[day]["points"] + scrum_days[previous_day]["integrated points"]
# --------------------------------------------------

### PLANEJAMENTO POR PROJETO

In [15]:
# --------------------------------------------------
# GRAFICO DONUTS - PERCENTUAL
# --------------------------------------------------  
labels = list(projetos.keys())
pts = []
for proj in projetos.keys():
    pts.append(int(projetos[proj]["totalPoints"]))


fig = go.Figure(data=[go.Pie(labels=labels, values=pts)])
fig.update_traces(hole=.0,hoverinfo='label+value')
fig.update_layout(
    title_text="Pontos por projeto - SEI - Sprint {}/{}".format(SPRINT, HOJE[-4:]),
    title_x=0.5,
    annotations=[dict(text='', x=0.50, y=0.5, font_size=20, showarrow=False)])
fig.show()

plotly.offline.plot(fig, filename = 'Sprint{}/pieChart_Projetos_SEI_Sprint_{}_{}.html'.format(SPRINT, SPRINT, HOJE[-4:]), auto_open=False)


'Sprint04/pieChart_Projetos_SEI_Sprint_04_2023.html'

### Plots

In [16]:
# --------------------------------------------------
# PLOTS
# --------------------------------------------------    
plot_data = {"date":[], "burnup":[], "burndown":[], "daily":[]}
for day in scrum_days.keys():
    plot_data["date"].append(day)
    plot_data["burnup"].append(scrum_days[day]["integrated points"])
    plot_data["burndown"].append(int(totalPontos - scrum_days[day]["integrated points"]))
    plot_data["daily"].append(int(scrum_days[day]["points"]))

    
# --------------------------------------------------
# BURNDOWN
# --------------------------------------------------  
fig = go.Figure()
fig.add_trace(go.Scatter(x=[startday, endday], y=[totalPontos, 0],
                    mode='lines',
                    name='ideal'))
fig.add_trace(go.Scatter(x=plot_data["date"], y=plot_data["burndown"],
                    mode='lines+markers',
                    name='Pontos a concluir'))
fig.update_layout(
    title={
        'text': "Burndown Completo - Grupo SEI - Sprint {}/{}".format(SPRINT, HOJE[-4:]),
        'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'},
    yaxis_title="Pontos restantes",
)
fig.show()
plotly.offline.plot(fig, filename = 'Sprint{}/burndown_SEI_Sprint_{}_{}.html'.format(SPRINT, SPRINT, HOJE[-4:]), auto_open=False)


# --------------------------------------------------
# BURNUP
# --------------------------------------------------  
fig = go.Figure()
fig.add_trace(go.Scatter(x=[startday, endday], y=[0, totalPontos],
                    mode='lines',
                    name='ideal'))
fig.add_trace(go.Scatter(x=plot_data["date"], y=plot_data["burnup"],
                    mode='lines+markers',
                    name='Pontos já queimados'))
fig.update_layout(
    title={
        'text': "Burnup Completo - Grupo SEI - Sprint {}/{}".format(SPRINT, HOJE[-4:]),
        'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'},
    yaxis_title="Pontos restantes",
)
fig.show()
plotly.offline.plot(fig, filename = 'Sprint{}/burnup_SEI_Sprint_{}_{}.html'.format(SPRINT, SPRINT, HOJE[-4:]), auto_open=False)


'Sprint04/burnup_SEI_Sprint_04_2023.html'

### Gráficos de pizza/donuts

In [17]:
# --------------------------------------------------
# GRAFICO DONUTS - PERCENTUAL
# --------------------------------------------------  
labels = list(status.keys())
pts = []
atv = []
for stat in status.keys():
    pts.append(int(status[stat]["pontos"]))
    atv.append(int(status[stat]["natividades"]))

# Create subplots: use 'domain' type for Pie subplot
fig = make_subplots(rows=1, cols=2, specs=[[{'type':'domain'}, {'type':'domain'}]])
fig.add_trace(go.Pie(labels=labels, values=pts, name="Pontos"),
              1, 1)
fig.add_trace(go.Pie(labels=labels, values=atv, name="Atividades"),
              1, 2)

# Use `hole` to create a donut-like pie chart
fig.update_traces(hole=.4, hoverinfo="label+percent+name")
fig.update_layout(
    title_text="Status Geral - Grupo SEI - Sprint {}/{}            ({})".format(SPRINT, HOJE[-4:], HOJE),
    title_x=0.2,
    # Add annotations in the center of the donut pies.
    annotations=[dict(text='Pontos', x=0.18, y=0.5, font_size=20, showarrow=False),
                 dict(text='Atividades', x=0.84, y=0.5, font_size=20, showarrow=False)])
fig.show()

plotly.offline.plot(fig, filename = 'Sprint{}/pieChart_SEI_Sprint_{}_{}.html'.format(SPRINT, SPRINT, HOJE[-4:]), auto_open=False)


'Sprint04/pieChart_SEI_Sprint_04_2023.html'

### Números

In [18]:
info = ""

info += ("------------------------------------------------------------\n")
info += ("*****  SPRINT {}/{}         ({})  *****\n".format(SPRINT, HOJE[-4:], HOJE))
info += ("------------------------------------------------------------\n")
info += ("\n\n\n") 

info += ("------------------------------------------------------------\n")
info += ("TOTAL DE PROJETOS ENT: {}\n\n".format(len(projetos)))
for proj in projetos:
    info += ("{}\n".format(projetos[proj]["name"]))
info += ("------------------------------------------------------------\n")


info += ("\n\n")  
info += ("------------------------------------------------------------\n")
info += ("TOTAL DE PONTOS\n")
info += ("PLANEJADOS: {}\n".format(int(totalPontos)))
info += ("{:.2f} pontos/dia\n".format(totalPontos/DIAS_UTEIS))
info += ("{:.2f} pontos/dia/pessoa\n".format(totalPontos/13/len(membros.keys())))
info += ("------------------------------------------------------------\n")



info += ("\n\n")  
info += ("------------------------------------------------------------\n")
info += ("STATUS\t\t#PONTOS\t\t% PONTOS\t#ATIVIDADES\t% ATIVIDADES\n")
for stat in status.keys():
    info += ("{}\t{}\t\t{:.1f}\t\t{}\t\t{:.1f}\n".format(stat.ljust(8, " "), int(status[stat]["pontos"]), 100*status[stat]["pontos"]/totalPontos,int(status[stat]["natividades"]), 100*status[stat]["natividades"]/len(atividade)))
info += ("Total\t\t{}\t\t\t\t{}\n".format(int(totalPontos), len(atividade)))
info += ("------------------------------------------------------------")


info += ("\n\n")  
info += ("------------------------------------------------------------\n")
info += ("MEMBRO\t\t#PONTOS\t\t#ATIVIDADES\n")
for pessoa in membros.keys():
    info += ("{}\t{}\t\t{}\n".format(pessoa.ljust(8, " "), int(membros[pessoa]["pontos"]), membros[pessoa]["natividades"]))
info += ("------------------------------------------------------------\n")



print(info)

# --------------------------------------------------
# Salvar em arquivo
# Se ultimo dia do sprint: salva em arquivo final
# Se dia intermediario, salva/sobrescreve arquivo de acompanhamento diario
# --------------------------------------------------
if (today <= endday):
    f = open("Sprint{}/overview/Sprint_{}_{}.txt".format(SPRINT, SPRINT, HOJE), "w")
    f.write(info)
    f.close()
# --------------------------------------------------

------------------------------------------------------------
*****  SPRINT 04/2023         (14-03-2023)  *****
------------------------------------------------------------



------------------------------------------------------------
TOTAL DE PROJETOS ENT: 9

ENT002.001 - MANUTENÇÃO E OPERAÇÃO DE SUBSISTEMAS DOS ACELERADORES DO SIRIUS
ENT002.002 - MANUTENÇÃO E OPERAÇÃO DOS GRUPOS ENT
ENT004.003 - SIMAR – SISTEMA DE MONITORAÇÃO E ATUAÇÃO EM RACKS
ENT004.022 - PROJETO DE NOVO NLK PARA TOP-UP
ENT004.039 - CONTROLE E SUPERVISÃO DAS FONTES DE RACKS DE BPMS
ENT034.001 - Projeto de um Wavelengthshifter - WLS
ENT094.003 - INTERF. P/ BIOSSENSOR NANOESTRUTURADO P/ MEDIDA NÃO INVAS. GLICEMIA
ENT104.002 - EVOLVER - HARDWARE PARA EVOLUÇÃO DE MICRORGANISMOS
ENT - Suporte e Prestação de Serviço
------------------------------------------------------------


------------------------------------------------------------
TOTAL DE PONTOS
PLANEJADOS: 326
25.08 pontos/dia
3.13 pontos/dia/pessoa
-----------